<a href="https://colab.research.google.com/github/Shigeru-Takaya/git_lesson/blob/master/ds_sample_1_LinearReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 必要なライブラリをインポート

In [ ]:
# サンプルデータセットが用意されているライブラリ sklearn.datasets から
# ボストンの住宅価格データを取得するためのメソッド load_boston をインポート
from sklearn.datasets import load_boston

# pandas のインポート
import pandas as pd

# 機械学習用ライブラリ sklearn（scikit-learn）内にあるライブラリ  から
# モデル構築（訓練用）/検証データ分割用メソッド train_test_split をインポート
from sklearn.model_selection import train_test_split

# 機械学習用ライブラリ sklearn（scikit-learn）から線形回帰用クラス linear_model をインポート 
from sklearn import linear_model

# 機械学習用ライブラリ sklearn（scikit-learn）内にあるライブラリ preprocessing から
# 標準化用クラス StandardScaler をインポート 
from sklearn.preprocessing import StandardScaler

# 統計解析用ライブラリ statsmodels 内にあるライブラリ stats.outliers_influence から
# 分散拡大係数（VIF）計算用メソッド variance_inflation_factor をインポート
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 精度評価指標を計算するためのメソッドをインポート
#   ・r2_score：決定係数
#   ・mean_squared_error：平均二乗誤差
#   ・mean_absolute_error：平均絶対誤差
#   ・median_absolute_error：Median Absolute Error
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error

# オブジェクトのコピーを行うためのモジュール copy をインポート
import copy

# 数学的な関数を使うためのライブラリ math をインポート
import math

# グラフ描画用ライブラリ matplotlib、seaborn をインポート
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 分散拡大係数（VIF）を確認するための関数を定義しておく

In [ ]:
# 投入したデータセットの全ての変数についてVIFを計算する関数 checkVIF の定義
def checkVIF( ExplanatoryVarDataSet ):
  tmp_columnList = ExplanatoryVarDataSet.columns
  vifList = []
  for i in range(len(tmp_columnList)):
    colname = tmp_columnList[i]
    vif = variance_inflation_factor(ExplanatoryVarDataSet.values, i)
    vifList.append( [ colname, vif ] )
  return  pd.DataFrame( vifList, columns=["COLUMN","VIF"] )

# AIC によるステップワイズ法で重回帰分析を実行するために必要な関数を定義しておく

In [ ]:
# AICを計算する関数 calcAIC
def calcAIC(x, y_data, y_predict):
  colNum = len(x.columns)
  rowNum = x.count()[0]
  y_data = y_data.values
  RSS = ( ( y_data - y_predict ) * ( y_data - y_predict ) ).sum()
  AIC = rowNum * ( math.log( 2 * math.pi * RSS / rowNum ) + 1 ) + 2 * ( colNum + 2 )
  return AIC

In [ ]:
# AICを基準としたステップワイズ法で変数選択された重回帰分析を実行する関数 LR_stepwise_AIC
def LR_stepwise_AIC(X, y, convergence=0.01, maxIter=100):
  # 回帰分析のためのインスタンスをつくる
  reg = linear_model.LinearRegression()
  returnReg = linear_model.LinearRegression()
  # 最終モデルの項目リストを定義しておく
  orgColList = X.columns
  modelColList = []
  maxR2modelColList = []
  modelCoef = 0
  modelIntercept = 0
  modelR2 = 0
  modelAIC = 1000000000
  residueColList = copy.copy(orgColList)
  # 
  count = 0
  while count < maxIter:
    count = count + 1
    for i in range(len(residueColList)):
      tmpColList = copy.copy(modelColList)
      if residueColList[i] in tmpColList:
        if len(tmpColList) > 1:
          tmpColList.remove( residueColList[i] )
        else:
          continue
      else:
        tmpColList.append( residueColList[i] )
      tmp_X = X.loc[:,tmpColList]
      reg.fit(tmp_X, y)
      if modelAIC > calcAIC(tmp_X, y, reg.predict(tmp_X) ):
        maxR2modelColList = copy.copy(tmpColList)
        modelR2 = reg.score(tmp_X, y)
        modelAIC = calcAIC(tmp_X, y, reg.predict(tmp_X) )
        print("model update: variables:" + str(maxR2modelColList) + ", AIC: " + str(modelAIC) )
        returnReg = copy.copy(reg)
    modelColList = copy.copy(maxR2modelColList)
  return returnReg, modelColList

# データの読み込み

In [ ]:
# ボストン住宅価格データを読み込む
loadBoston = load_boston()
boston = pd.DataFrame(loadBoston.data, columns = loadBoston.feature_names)
boston["MEDV"] = loadBoston.target
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


# 全データをモデル構築用データ、モデル検証用データに分割する

In [ ]:
# 目的変数と説明変数に分割
columnList = boston.columns.values.tolist()
columnList.remove("MEDV")
X = boston.loc[:,columnList]
y = boston.loc[:,["MEDV"]]

# モデル構築用データ、モデル検証用データに分割（70:30に分割）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(354, 13)
(152, 13)
(354, 1)
(152, 1)


# 標準化の実施

In [ ]:
# X_train のデータを使い標準化パラメータを獲得してから、X_train、X_test に対して標準化を実施
scaler_X = StandardScaler()
scaler_X.fit( X_train )
X_train_std = pd.DataFrame(scaler_X.transform(X_train), columns=columnList)
X_test_std = pd.DataFrame(scaler_X.transform(X_test), columns=columnList)

In [ ]:
# y_train のデータを使い標準化パラメータを獲得してから、y_train、y_test に対して標準化を実施
scaler_y = StandardScaler()
scaler_y.fit( y_train )
y_train_std = pd.DataFrame(scaler_y.transform(y_train), columns=["MEDV"])
y_test_std = pd.DataFrame(scaler_y.transform(y_test), columns=["MEDV"])

# AICによるステップワイズ法で変数選択をしつつ重回帰分析を実行する

In [ ]:
reg, colList = LR_stepwise_AIC(X_train_std, y_train_std, convergence=0.01, maxIter=30)

model update: variables:['CRIM'], AIC: 955.2588842428581
model update: variables:['INDUS'], AIC: 926.6232441904497
model update: variables:['RM'], AIC: 773.7065705544326
model update: variables:['LSTAT'], AIC: 721.4519714451382
model update: variables:['LSTAT', 'CRIM'], AIC: 718.0852548020802
model update: variables:['LSTAT', 'CHAS'], AIC: 705.027599959596
model update: variables:['LSTAT', 'RM'], AIC: 642.3464005438564
model update: variables:['LSTAT', 'RM', 'CRIM'], AIC: 634.1861163662331
model update: variables:['LSTAT', 'RM', 'CHAS'], AIC: 629.1735966481272
model update: variables:['LSTAT', 'RM', 'PTRATIO'], AIC: 605.6717543684476
model update: variables:['LSTAT', 'RM', 'PTRATIO', 'CRIM'], AIC: 602.8723312545401
model update: variables:['LSTAT', 'RM', 'PTRATIO', 'CHAS'], AIC: 592.7242306478062
model update: variables:['LSTAT', 'RM', 'PTRATIO', 'CHAS', 'CRIM'], AIC: 589.6755654228307
model update: variables:['LSTAT', 'RM', 'PTRATIO', 'CHAS', 'DIS'], AIC: 584.6355596001058
model updat

In [ ]:
# 分析結果として、回帰係数（reg.coef_）、切片（reg.intercept_）を表示する
print(colList)
print(reg.coef_[0])
print(reg.intercept_[0])

['LSTAT', 'RM', 'PTRATIO', 'CHAS', 'B', 'DIS', 'NOX', 'CRIM', 'RAD', 'ZN', 'TAX']
[-0.421386    0.30266284 -0.21805609  0.08813774  0.10745286 -0.30868068
 -0.19125411 -0.11929884  0.21377563  0.08556997 -0.13217125]
-1.226626513824023e-16


In [ ]:
# 多重共線性を確認する
vif = checkVIF(X_train_std[colList])
vif["COEF"] = reg.coef_[0]
vif

,COLUMN,VIF,COEF
0,LSTAT,2.480300,-0.421386
1,RM,1.811865,0.302663
2,PTRATIO,1.792094,-0.218056
3,CHAS,1.088475,0.088138
4,B,1.260822,0.107453
5,DIS,3.525581,-0.308681
6,NOX,3.931789,-0.191254
7,CRIM,1.728729,-0.119299
8,RAD,6.916102,0.213776
9,ZN,2.289778,0.085570


In [ ]:
# モデル構築用データについて各精度評価指標を計算する
print("R2 SCORE:" + str(round(r2_score(y_train_std, reg.predict(X_train_std[colList])),4)) )
print("MSE     :" + str(round(mean_squared_error(y_train_std, reg.predict(X_train_std[colList])),4)) )
print("MAE     :" + str(round(mean_absolute_error(y_train_std, reg.predict(X_train_std[colList])),4)) )
print("MedianAE:" + str(round(median_absolute_error(y_train_std, reg.predict(X_train_std[colList])),4)) )

R2 SCORE:0.7428
MSE     :0.2572
MAE     :0.359
MedianAE:0.2652


In [ ]:
# モデル検証用データについて各精度評価指標を計算する
print("R2 SCORE:" + str(round(r2_score(y_test_std, reg.predict(X_test_std[colList])),4)) )
print("MSE     :" + str(round(mean_squared_error(y_test_std, reg.predict(X_test_std[colList])),4)) )
print("MAE     :" + str(round(mean_absolute_error(y_test_std, reg.predict(X_test_std[colList])),4)) )
print("MedianAE:" + str(round(median_absolute_error(y_test_std, reg.predict(X_test_std[colList])),4)) )

R2 SCORE:0.7149
MSE     :0.2417
MAE     :0.3321
MedianAE:0.2531
